<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Notebook: Qwen Image-Edit 2.5 - API ComfyUI

**Objectif**: Maîtriser génération et édition d'images via API Qwen-Image-Edit (backend ComfyUI)

## 🎯 Ce que vous allez apprendre

1. Différence API **Forge** (simple) vs **ComfyUI** (workflows JSON)
2. Pattern **"queue and poll"** pour génération asynchrone
3. Création workflows **Text-to-Image** et **Image-to-Image**
4. Optimisation paramètres (**steps**, **cfg**, **denoise**)
5. Troubleshooting erreurs courantes (**timeout**, **CUDA OOM**)

## 🚀 API Qwen-Image-Edit

| Caractéristique | Valeur |
|----------------|--------|
| **URL Production** | `https://qwen-image-edit.myia.io` |
| **Modèle** | Qwen-Image-Edit-2509-FP8 (54GB) |
| **GPU** | RTX 3090 (24GB VRAM) |
| **Latence Typique** | 5-10 secondes |
| **Résolution Optimale** | 512x512 pixels |

## 🔍 ComfyUI vs Forge

**Forge (SD XL Turbo)**:
- ✅ API simple (1 requête POST)
- ✅ Ultra-rapide (1-3s)
- ❌ Pas d'édition images
- ❌ Moins flexible

**ComfyUI (Qwen)**:
- ✅ Workflows JSON complexes
- ✅ Édition images avancée
- ✅ Contrôle fin (28 custom nodes)
- ❌ API plus complexe (queue + poll)

**Recommandation**: Commencer avec Forge pour prototypes, affiner avec Qwen pour production.

## 📚 Prérequis

```bash
pip install requests pillow matplotlib
```

**Temps estimé**: 90-120 minutes

In [1]:
# Imports standard
import requests
import json
import base64
import time
import uuid
import os
from typing import Dict, Optional, List
from io import BytesIO

# Visualisation
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Configuration environnement
from dotenv import load_dotenv
load_dotenv()

# Configuration API
API_BASE_URL = "https://qwen-image-edit.myia.io"
CLIENT_ID = str(uuid.uuid4())  # ID unique pour tracking

# Authentification - Token ComfyUI standardisé
COMFYUI_API_TOKEN = os.getenv("COMFYUI_API_TOKEN")

if not COMFYUI_API_TOKEN:
    print("⚠️  COMFYUI_API_TOKEN non trouvé - connexion sans authentification")
    print("\n📋 Pour activer l'authentification :")
    print("   1. Créez un fichier .env dans MyIA.AI.Notebooks/GenAI/")
    print("   2. Ajoutez : COMFYUI_API_TOKEN=votre_token_ici")
    print("   3. Obtenez le token via scripts/genai-auth/extract-bearer-tokens.ps1")
    print("\n✓ Le notebook fonctionnera en mode dégradé (si serveur non sécurisé)\n")
    COMFYUI_API_TOKEN = None
else:
    print("✅ Configuration chargée")
    print(f"📡 API: {API_BASE_URL}")
    print(f"🆔 Client ID: {CLIENT_ID}")
    print(f"🔐 Token: {COMFYUI_API_TOKEN[:15]}...{COMFYUI_API_TOKEN[-5:]}" if len(COMFYUI_API_TOKEN) > 20 else f"🔐 Token: (court)")

✅ Configuration chargée
📡 API: https://qwen-image-edit.myia.io
🆔 Client ID: 010e4ab5-a332-4786-99cb-896b5e983d9e
🔐 Token: $2b$12$UDceblhZ...6coni


## 🏗️ Architecture ComfyUI: Workflows JSON

### Différence fondamentale avec Forge

**API Forge (POST direct)**:
```python
response = requests.post(url, json={"prompt": "astronaut"})
image_base64 = response.json()["image"]
```

**API ComfyUI (Queue + Poll)**:
```python
# 1. Soumettre workflow JSON
response = requests.post(f"{url}/prompt", json={
    "prompt": workflow_json,
    "client_id": client_id
})
prompt_id = response.json()["prompt_id"]

# 2. Attendre complétion (polling)
while True:
    history = requests.get(f"{url}/history/{prompt_id}")
    if history.json().get(prompt_id, {}).get("status", {}).get("completed"):
        break
    time.sleep(1)

# 3. Récupérer images
images = history.json()[prompt_id]["outputs"]
```

### Structure Workflow ComfyUI

Un **workflow** est un **graph JSON** de **nodes connectés**:

```json
{
  "1": {  // Node Checkpoint Loader
    "class_type": "CheckpointLoaderSimple",
    "inputs": {"ckpt_name": "qwen-image-edit.safetensors"}
  },
  "2": {  // Node Sampler
    "class_type": "KSampler",
    "inputs": {
      "model": ["1", 0],  // Connexion: output du node 1
      "steps": 20,
      "cfg": 7.0,
      "seed": 42
    }
  },
  "3": {  // Node Save Image
    "class_type": "SaveImage",
    "inputs": {"images": ["2", 0]}
  }
}
```

**Workflow = Pipeline modulaire** où chaque node effectue une opération (charger modèle, sampler, encoder texte, etc.).

### Anatomie d'un Node

| Propriété | Description | Exemple |
|-----------|-------------|----------|
| **`class_type`** | Type de node ComfyUI | `"CLIPTextEncode"` |
| **`inputs`** | Paramètres du node | `{"text": "astronaut"}` |
| **Connexions** | `[node_id, output_slot]` | `["5", 0]` |

**28 custom nodes** disponibles pour Qwen (voir documentation complète dans guide étudiants).

### 🔧 Visualisation Architecture Workflow ComfyUI

**Diagramme ASCII d'un workflow ComfyUI typique**:

```
┌─────────────────────────────────────────────────────────────┐
│                    WORKFLOW COMFYUI                         │
│                                                             │
│  ┌──────────────┐        ┌──────────────┐                 │
│  │ Load Model   │───────▶│ CLIP Text    │                 │
│  │ (Checkpoint) │        │ Encode       │                 │
│  └──────────────┘        └──────────────┘                 │
│         │                       │                          │
│         │                       ▼                          │
│         │                ┌──────────────┐                 │
│         │                │ Conditioning │                 │
│         │                │ (Prompts)    │                 │
│         │                └──────────────┘                 │
│         │                       │                          │
│         ▼                       ▼                          │
│  ┌──────────────────────────────────────┐                │
│  │          KSampler                     │                │
│  │  (steps, denoise, seed, sampler)     │                │
│  └──────────────────────────────────────┘                │
│                    │                                       │
│                    ▼                                       │
│             ┌──────────────┐                              │
│             │ VAE Decode   │                              │
│             └──────────────┘                              │
│                    │                                       │
│                    ▼                                       │
│             ┌──────────────┐                              │
│             │ Save Image   │                              │
│             └──────────────┘                              │
└─────────────────────────────────────────────────────────────┘
```

**Flux de données**:
1. **Checkpoint** → Fournit le modèle (model, CLIP, VAE)
2. **CLIP Text Encode** → Convertit le prompt en embeddings
3. **KSampler** → Génère l'image latente à partir des embeddings
4. **VAE Decode** → Convertit l'image latente en image RGB
5. **Save Image** → Sauvegarde l'image finale

**Correspondance JSON**:
```json
{
  "1": {"class_type": "CheckpointLoaderSimple", ...},
  "2": {"class_type": "CLIPTextEncode", "inputs": {"clip": ["1", 1], ...}},
  "3": {"class_type": "KSampler", "inputs": {"model": ["1", 0], ...}},
  "4": {"class_type": "VAEDecode", "inputs": {"samples": ["3", 0], ...}},
  "5": {"class_type": "SaveImage", "inputs": {"images": ["4", 0]}}
}
```

**Notation `["ID_NODE", INDEX_OUTPUT]`**:
- `["1", 0]` = Output 0 (model) du node 1
- `["1", 1]` = Output 1 (CLIP) du node 1
- `["3", 0]` = Output 0 (latents) du node 3

💡 **Astuce**: Chaque node peut avoir plusieurs outputs. L'index détermine lequel utiliser.

In [2]:
class ComfyUIClient:
    """Client pédagogique API ComfyUI pour Qwen avec authentification Bearer"""
    
    def __init__(self, base_url=API_BASE_URL, client_id=CLIENT_ID, auth_token=None):
        self.base_url = base_url
        self.client_id = client_id
        self.session = requests.Session()
        
        # Configuration authentification
        if auth_token:
            self.session.headers.update({
                "Authorization": f"Bearer {auth_token}"
            })
        elif COMFYUI_API_TOKEN:  # Utilise variable globale si disponible
            self.session.headers.update({
                "Authorization": f"Bearer {COMFYUI_API_TOKEN}"
            })
        # Pas d'erreur si pas de token - graceful degradation
    
    def execute_workflow(
        self, 
        workflow_json: Dict, 
        wait_for_completion: bool = True,
        max_wait: int = 120,
        verbose: bool = True
    ) -> Dict:
        """Exécute workflow ComfyUI et récupère résultats
        
        Args:
            workflow_json: Workflow ComfyUI (dict)
            wait_for_completion: Attendre fin génération
            max_wait: Timeout en secondes
            verbose: Afficher logs progression
        
        Returns:
            dict: {"prompt_id", "outputs", "status"}
        """
        # 1. Soumettre workflow
        if verbose:
            print("📤 Soumission workflow...")
        
        response = self.session.post(
            f"{self.base_url}/prompt",
            json={"prompt": workflow_json, "client_id": self.client_id}
        )
        response.raise_for_status()
        prompt_id = response.json()["prompt_id"]
        
        if verbose:
            print(f"✅ Workflow queued: {prompt_id}")
        
        if not wait_for_completion:
            return {"prompt_id": prompt_id, "status": "queued"}
        
        # 2. Polling complétion
        start_time = time.time()
        while True:
            elapsed = time.time() - start_time
            if elapsed > max_wait:
                raise TimeoutError(f"Timeout après {max_wait}s")
            
            history_response = self.session.get(
                f"{self.base_url}/history/{prompt_id}"
            )
            history_response.raise_for_status()
            history = history_response.json()
            
            if prompt_id not in history:
                time.sleep(1)
                continue
            
            prompt_data = history[prompt_id]
            status = prompt_data.get("status", {})
            
            if status.get("completed"):
                if verbose:
                    print(f"✅ Complet en {elapsed:.1f}s")
                
                # 3. Extraire outputs
                outputs = prompt_data.get("outputs", {})
                images = []
                
                for node_id, node_output in outputs.items():
                    if "images" in node_output:
                        for img_info in node_output["images"]:
                            # Récupérer image
                            img_response = self.session.get(
                                f"{self.base_url}/view",
                                params={
                                    "filename": img_info["filename"],
                                    "subfolder": img_info.get("subfolder", ""),
                                    "type": img_info.get("type", "output")
                                }
                            )
                            img_response.raise_for_status()
                            images.append({
                                "data": img_response.content,
                                "filename": img_info["filename"]
                            })
                
                return {
                    "prompt_id": prompt_id,
                    "status": "completed",
                    "outputs": outputs,
                    "images": images,
                    "duration": elapsed
                }
            
            if status.get("status_str") == "error":
                error_msg = status.get("messages", ["Unknown error"])
                raise RuntimeError(f"ComfyUI error: {error_msg}")
            
            if verbose and int(elapsed) % 5 == 0:
                print(f"⏳ En cours... ({elapsed:.0f}s)")
            
            time.sleep(1)
    
    def display_images(self, result: Dict, figsize=(12, 4)):
        """Affiche images résultats"""
        images = result.get("images", [])
        if not images:
            print("⚠️ Aucune image générée")
            return
        
        n = len(images)
        fig, axes = plt.subplots(1, n, figsize=figsize)
        if n == 1:
            axes = [axes]
        
        for ax, img_data in zip(axes, images):
            img = Image.open(BytesIO(img_data["data"]))
            ax.imshow(img)
            ax.set_title(img_data["filename"], fontsize=10)
            ax.axis("off")
        
        plt.tight_layout()
        plt.show()

# Instancier client avec authentification
client = ComfyUIClient(auth_token=COMFYUI_API_TOKEN)
if COMFYUI_API_TOKEN:
    print("✅ ComfyUIClient prêt avec authentification")
else:
    print("⚠️  ComfyUIClient prêt en mode sans authentification")

✅ ComfyUIClient prêt avec authentification


## 🚀 Workflow Minimal: "Hello World"

### Objectif

Créer le workflow le plus simple possible pour **valider l'API** et comprendre la structure JSON.

### Workflow Text-to-Image Basique

**Pipeline**:
1. **Load Checkpoint** → Charger modèle Qwen
2. **CLIP Text Encode** → Encoder prompt texte
3. **Empty Latent Image** → Créer canvas vide
4. **KSampler** → Générer image
5. **VAE Decode** → Convertir latent → pixels
6. **Save Image** → Sauvegarder résultat

### Paramètres Critiques

| Paramètre | Valeur | Impact |
|-----------|--------|--------|
| **steps** | 20 | Qualité (↑ steps = ↑ qualité, ↑ temps) |
| **cfg** | 7.0 | Fidélité prompt (7-9 optimal) |
| **sampler** | euler | Algorithme génération |
| **scheduler** | normal | Stratégie steps |
| **denoise** | 1.0 | Force génération (1.0 = 100%) |
| **seed** | 42 | Reproductibilité |

**Temps attendu**: 5-10 secondes (512x512)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
# Workflow Text-to-Image minimal
workflow_hello = {
    "1": {  # Load Checkpoint
        "class_type": "CheckpointLoaderSimple",
        "inputs": {
            "ckpt_name": "qwen-image-edit-2509-fp8.safetensors"
        }
    },
    "2": {  # CLIP Text Encode (prompt positif)
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "a majestic astronaut floating in space, photorealistic, 8k, detailed",
            "clip": ["1", 1]  # Connexion: output CLIP du checkpoint
        }
    },
    "3": {  # CLIP Text Encode (prompt négatif)
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "blurry, low quality, distorted",
            "clip": ["1", 1]
        }
    },
    "4": {  # Empty Latent Image (canvas)
        "class_type": "EmptyLatentImage",
        "inputs": {
            "width": 512,
            "height": 512,
            "batch_size": 1
        }
    },
    "5": {  # KSampler (génération)
        "class_type": "KSampler",
        "inputs": {
            "model": ["1", 0],  # Model du checkpoint
            "positive": ["2", 0],  # Prompt positif
            "negative": ["3", 0],  # Prompt négatif
            "latent_image": ["4", 0],  # Canvas latent
            "seed": 42,
            "steps": 20,
            "cfg": 7.0,
            "sampler_name": "euler",
            "scheduler": "normal",
            "denoise": 1.0
        }
    },
    "6": {  # VAE Decode
        "class_type": "VAEDecode",
        "inputs": {
            "samples": ["5", 0],  # Latent du sampler
            "vae": ["1", 2]  # VAE du checkpoint
        }
    },
    "7": {  # Save Image
        "class_type": "SaveImage",
        "inputs": {
            "images": ["6", 0],  # Pixels du VAE
            "filename_prefix": "ComfyUI"
        }
    }
}

# Exécution
print("🚀 Lancement génération...")
result = client.execute_workflow(workflow_hello, verbose=True)

# Affichage
client.display_images(result)
print(f"\n✅ Image générée en {result['duration']:.1f}s")

🚀 Lancement génération...
📤 Soumission workflow...


HTTPError: 400 Client Error: Bad Request for url: https://qwen-image-edit.myia.io/prompt

In [ ]:
# ========================================
# 🎯 WORKFLOW RÉEL 1: Édition Simple Image
# ========================================

def create_simple_edit_workflow(image_name: str, edit_prompt: str, denoise: float = 0.5) -> dict:
    """Workflow édition simple d'une image existante
    
    Args:
        image_name: Nom du fichier image uploadé sur ComfyUI
        edit_prompt: Description de l'édition souhaitée
        denoise: Force de l'édition (0.0 = aucune, 1.0 = complète)
    
    Returns:
        Workflow JSON prêt à exécuter
    """
    workflow = {
        "1": {
            "class_type": "CheckpointLoaderSimple",
            "inputs": {"ckpt_name": "qwen_vl_model.safetensors"}
        },
        "2": {
            "class_type": "LoadImage",
            "inputs": {"image": image_name}
        },
        "3": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "text": edit_prompt,
                "clip": ["1", 1]
            }
        },
        "4": {
            "class_type": "VAEEncode",
            "inputs": {
                "pixels": ["2", 0],
                "vae": ["1", 2]
            }
        },
        "5": {
            "class_type": "KSampler",
            "inputs": {
                "seed": 42,
                "steps": 20,
                "cfg": 7.0,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": denoise,
                "model": ["1", 0],
                "positive": ["3", 0],
                "negative": ["3", 0],  # Pas de prompt négatif pour édition simple
                "latent_image": ["4", 0]
            }
        },
        "6": {
            "class_type": "VAEDecode",
            "inputs": {
                "samples": ["5", 0],
                "vae": ["1", 2]
            }
        },
        "7": {
            "class_type": "SaveImage",
            "inputs": {
                "images": ["6", 0],
                "filename_prefix": "qwen_edit_simple"
            }
        }
    }
    return workflow

# ========================================
# 🎯 WORKFLOW RÉEL 2: Chaînage Nodes Avancé
# ========================================

def create_chained_workflow(base_prompt: str, refine_prompt: str) -> dict:
    """Workflow avec chaînage: génération base + raffinement
    
    Architecture:
        1. Génération image base (text-to-image)
        2. Raffinement avec nouveau prompt (image-to-image)
    
    Args:
        base_prompt: Prompt initial génération
        refine_prompt: Prompt raffinement/amélioration
    
    Returns:
        Workflow JSON avec 2 étapes KSampler
    """
    workflow = {
        # Étape 1: Génération base
        "1": {
            "class_type": "CheckpointLoaderSimple",
            "inputs": {"ckpt_name": "qwen_vl_model.safetensors"}
        },
        "2": {
            "class_type": "CLIPTextEncode",
            "inputs": {"text": base_prompt, "clip": ["1", 1]}
        },
        "3": {
            "class_type": "EmptyLatentImage",
            "inputs": {"width": 512, "height": 512, "batch_size": 1}
        },
        "4": {
            "class_type": "KSampler",
            "inputs": {
                "seed": 42,
                "steps": 10,
                "cfg": 7.0,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1.0,  # Génération complète
                "model": ["1", 0],
                "positive": ["2", 0],
                "negative": ["2", 0],
                "latent_image": ["3", 0]
            }
        },
        # Étape 2: Raffinement
        "5": {
            "class_type": "CLIPTextEncode",
            "inputs": {"text": refine_prompt, "clip": ["1", 1]}
        },
        "6": {
            "class_type": "KSampler",
            "inputs": {
                "seed": 43,
                "steps": 10,
                "cfg": 7.0,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 0.3,  # Raffinement léger
                "model": ["1", 0],
                "positive": ["5", 0],
                "negative": ["5", 0],
                "latent_image": ["4", 0]  # Sortie de l'étape 1
            }
        },
        "7": {
            "class_type": "VAEDecode",
            "inputs": {"samples": ["6", 0], "vae": ["1", 2]}
        },
        "8": {
            "class_type": "SaveImage",
            "inputs": {
                "images": ["7", 0],
                "filename_prefix": "qwen_chained"
            }
        }
    }
    return workflow

# ========================================
# EXEMPLE D'UTILISATION
# ========================================

# Workflow 1: Édition simple
workflow_simple = create_simple_edit_workflow(
    image_name="cat.png",
    edit_prompt="Add sunglasses to the cat",
    denoise=0.5
)
print("✅ Workflow édition simple créé")
print(f"   Nodes: {len(workflow_simple)}")

# Workflow 2: Chaînage
workflow_chained = create_chained_workflow(
    base_prompt="A cat sitting on a chair",
    refine_prompt="High quality, professional photography, detailed fur"
)
print("✅ Workflow chaîné créé")
print(f"   Nodes: {len(workflow_chained)}")
print(f"   KSamplers: 2 (base + raffinement)")

# 💡 Pour exécuter ces workflows:
# client = ComfyUIClient("https://qwen-image-edit.myia.io")
# result = client.execute_workflow(workflow_simple)
# images = client.get_results(result["prompt_id"])

## 🖼️ Édition Images avec Qwen VLM

### Capacités Qwen Vision-Language Model

**Qwen-Image-Edit** combine:
- 🧠 **Vision Encoder** (CLIP-like) pour comprendre images
- ✍️ **Language Model** pour interpréter instructions texte
- 🎨 **Diffusion Model** pour éditer images

### Cas d'Usage Typiques

| Tâche | Exemple Prompt |
|-------|----------------|
| **Style Transfer** | `"Convert to watercolor painting"` |
| **Object Addition** | `"Add a red balloon in the sky"` |
| **Color Grading** | `"Make the image warmer, golden hour lighting"` |
| **Background Change** | `"Replace background with snowy mountains"` |
| **Detail Enhancement** | `"Enhance facial details, 8k quality"` |

### Pattern Image-to-Image

**Différence clé avec Text-to-Image**:
- **Nouveau node**: `LoadImage` pour charger image source
- **Paramètre critique**: `denoise` (0.0-1.0)
  - `denoise=0.1`: Édition subtile (retouche légère)
  - `denoise=0.5`: Édition modérée (style transfer)
  - `denoise=0.9`: Édition forte (reconstruction quasi-totale)

**Workflow**:
1. **Load Image** → Charger image source
2. **CLIP Vision Encode** → Encoder image
3. **CLIP Text Encode** → Encoder prompt édition
4. **VAE Encode** → Convertir pixels → latent
5. **KSampler** (denoise < 1.0) → Éditer latent
6. **VAE Decode** → Convertir latent → pixels
7. **Save Image** → Sauvegarder résultat

In [ ]:
def upload_image_to_comfyui(image_path: str) -> str:
    """Upload image vers ComfyUI pour édition
    
    Args:
        image_path: Chemin image locale ou URL
    
    Returns:
        str: Nom fichier uploadé dans ComfyUI
    """
    # Charger image
    if image_path.startswith('http'):
        response = requests.get(image_path)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(image_path)
    
    # Convertir en bytes
    img_bytes = BytesIO()
    img.save(img_bytes, format='PNG')
    img_bytes.seek(0)
    
    # Upload vers ComfyUI
    files = {'image': ('input.png', img_bytes, 'image/png')}
    response = client.session.post(
        f"{API_BASE_URL}/upload/image",
        files=files
    )
    response.raise_for_status()
    
    filename = response.json()['name']
    print(f"✅ Image uploadée: {filename}")
    return filename

# Test upload avec image exemple
# Note: Remplacer par votre propre image
test_image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg"

try:
    uploaded_filename = upload_image_to_comfyui(test_image_url)
    print(f"📁 Fichier: {uploaded_filename}")
except Exception as e:
    print(f"⚠️ Erreur upload: {e}")
    print("💡 Conseil: Utiliser une image locale ou vérifier URL")

## 🎨 Workflow Image-to-Image Complet

### Architecture

**Pipeline édition**:
1. **Load Image** → Charger image source uploadée
2. **VAE Encode** → Convertir pixels → latent space
3. **CLIP Text Encode** → Encoder instructions édition
4. **KSampler** (denoise partiel) → Éditer latent
5. **VAE Decode** → Reconvertir latent → pixels
6. **Save Image** → Sauvegarder résultat

### Paramètre Critique: denoise

**Impact sur édition**:

| denoise | Type Édition | Exemple |
|---------|-------------|----------|
| **0.1-0.3** | Retouche subtile | Correction couleurs, amélioration détails |
| **0.4-0.6** | Édition modérée | Style transfer, ajout éléments mineurs |
| **0.7-0.9** | Édition forte | Changement scène, reconstruction majeure |
| **1.0** | Génération totale | Ignore quasi totalement image source |

**Recommandation**: Commencer avec `denoise=0.5` et ajuster selon résultat.

In [ ]:
# Workflow Image-to-Image (édition)
workflow_img2img = {
    "1": {  # Load Checkpoint
        "class_type": "CheckpointLoaderSimple",
        "inputs": {
            "ckpt_name": "qwen-image-edit-2509-fp8.safetensors"
        }
    },
    "2": {  # Load Image (image source uploadée)
        "class_type": "LoadImage",
        "inputs": {
            "image": uploaded_filename  # Variable de cellule précédente
        }
    },
    "3": {  # VAE Encode (pixels → latent)
        "class_type": "VAEEncode",
        "inputs": {
            "pixels": ["2", 0],  # Image source
            "vae": ["1", 2]  # VAE du checkpoint
        }
    },
    "4": {  # CLIP Text Encode (prompt édition)
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "watercolor painting style, artistic, vibrant colors",
            "clip": ["1", 1]
        }
    },
    "5": {  # CLIP Text Encode (prompt négatif)
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "blurry, low quality, pixelated",
            "clip": ["1", 1]
        }
    },
    "6": {  # KSampler (édition avec denoise partiel)
        "class_type": "KSampler",
        "inputs": {
            "model": ["1", 0],
            "positive": ["4", 0],
            "negative": ["5", 0],
            "latent_image": ["3", 0],  # Latent de l'image source
            "seed": 42,
            "steps": 25,
            "cfg": 7.5,
            "sampler_name": "euler",
            "scheduler": "normal",
            "denoise": 0.5  # Édition modérée (50%)
        }
    },
    "7": {  # VAE Decode (latent → pixels)
        "class_type": "VAEDecode",
        "inputs": {
            "samples": ["6", 0],
            "vae": ["1", 2]
        }
    },
    "8": {  # Save Image
        "class_type": "SaveImage",
        "inputs": {
            "images": ["7", 0],
            "filename_prefix": "Qwen_Edit"
        }
    }
}

# Exécution (seulement si image uploadée précédemment)
try:
    print("🎨 Lancement édition image...")
    result = client.execute_workflow(workflow_img2img, verbose=True)
    
    # Affichage avant/après
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Image originale
    original_img = Image.open(BytesIO(client.session.get(
        f"{API_BASE_URL}/view",
        params={"filename": uploaded_filename, "type": "input"}
    ).content))
    axes[0].imshow(original_img)
    axes[0].set_title("Image Originale", fontsize=12)
    axes[0].axis("off")
    
    # Image éditée
    edited_img = Image.open(BytesIO(result["images"][0]["data"]))
    axes[1].imshow(edited_img)
    axes[1].set_title(f"Image Éditée (denoise=0.5)", fontsize=12)
    axes[1].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Édition complète en {result['duration']:.1f}s")
except NameError:
    print("⚠️ Exécutez d'abord la cellule d'upload d'image")
except Exception as e:
    print(f"❌ Erreur: {e}")

## 🔬 Expérimentation: Comparaison Denoise

### Objectif Pédagogique

Comprendre **impact du paramètre `denoise`** sur qualité édition.

### Méthodologie

1. Workflow identique (même prompt, seed, etc.)
2. Variation **uniquement** du paramètre `denoise`
3. Comparaison visuelle résultats

### Hypothèse

**denoise faible** (0.2) → Édition subtile, image proche de l'originale
**denoise moyen** (0.5) → Bon compromis édition/préservation
**denoise élevé** (0.8) → Édition forte, risque divergence

### Configuration Test

```python
denoise_values = [0.2, 0.5, 0.8]
prompt = "convert to dramatic black and white, high contrast"
seed = 42  # Fixe pour comparabilité
```

In [ ]:
# Comparaison denoise (denoise=0.2, 0.5, 0.8)
denoise_values = [0.2, 0.5, 0.8]
edit_prompt = "convert to dramatic black and white, high contrast"

results_denoise = []

for denoise_val in denoise_values:
    print(f"\n🔄 Test denoise={denoise_val}...")
    
    # Créer workflow avec denoise spécifique
    workflow_denoise_test = {
        "1": {
            "class_type": "CheckpointLoaderSimple",
            "inputs": {"ckpt_name": "qwen-image-edit-2509-fp8.safetensors"}
        },
        "2": {
            "class_type": "LoadImage",
            "inputs": {"image": uploaded_filename}
        },
        "3": {
            "class_type": "VAEEncode",
            "inputs": {
                "pixels": ["2", 0],
                "vae": ["1", 2]
            }
        },
        "4": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "text": edit_prompt,
                "clip": ["1", 1]
            }
        },
        "5": {
            "class_type": "CLIPTextEncode",
            "inputs": {
                "text": "blurry, low quality",
                "clip": ["1", 1]
            }
        },
        "6": {
            "class_type": "KSampler",
            "inputs": {
                "model": ["1", 0],
                "positive": ["4", 0],
                "negative": ["5", 0],
                "latent_image": ["3", 0],
                "seed": 42,
                "steps": 25,
                "cfg": 7.5,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": denoise_val  # Variable testée
            }
        },
        "7": {
            "class_type": "VAEDecode",
            "inputs": {
                "samples": ["6", 0],
                "vae": ["1", 2]
            }
        },
        "8": {
            "class_type": "SaveImage",
            "inputs": {
                "images": ["7", 0],
                "filename_prefix": f"Denoise_{denoise_val}"
            }
        }
    }
    
    try:
        result = client.execute_workflow(workflow_denoise_test, verbose=False)
        results_denoise.append({
            "denoise": denoise_val,
            "result": result
        })
        print(f"✅ Complété en {result['duration']:.1f}s")
    except Exception as e:
        print(f"❌ Erreur: {e}")
        results_denoise.append({"denoise": denoise_val, "error": str(e)})

# Affichage comparatif
if len(results_denoise) == 3 and all('result' in r for r in results_denoise):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, res in enumerate(results_denoise):
        img = Image.open(BytesIO(res["result"]["images"][0]["data"]))
        axes[i].imshow(img)
        axes[i].set_title(f"denoise={res['denoise']}", fontsize=12)
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Observations:")
    print("- denoise=0.2: Édition subtile, préserve détails originaux")
    print("- denoise=0.5: Équilibre édition/préservation")
    print("- denoise=0.8: Édition forte, peut diverger de l'original")
else:
    print("⚠️ Certains tests ont échoué, vérifiez les erreurs ci-dessus")

In [ ]:
# ========================================
# 🖼️ COMPARAISON AVANT/APRÈS: Side-by-Side
# ========================================

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from typing import List, Tuple

def compare_before_after(
    original_path: str,
    edited_path: str,
    title_original: str = "Image Originale",
    title_edited: str = "Image Éditée",
    show_metrics: bool = True
) -> None:
    """Affiche comparaison side-by-side avec métriques qualité
    
    Args:
        original_path: Chemin image originale
        edited_path: Chemin image éditée
        title_original: Titre image originale
        title_edited: Titre image éditée
        show_metrics: Afficher métriques qualité (PSNR, SSIM)
    """
    # Charger images
    img_original = Image.open(original_path)
    img_edited = Image.open(edited_path)
    
    # Convertir en numpy arrays
    arr_original = np.array(img_original)
    arr_edited = np.array(img_edited)
    
    # Créer figure avec 2 colonnes
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Image originale
    axes[0].imshow(arr_original)
    axes[0].set_title(f"{title_original}\n{img_original.size[0]}x{img_original.size[1]}", 
                      fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Image éditée
    axes[1].imshow(arr_edited)
    axes[1].set_title(f"{title_edited}\n{img_edited.size[0]}x{img_edited.size[1]}", 
                      fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Métriques qualité
    if show_metrics and arr_original.shape == arr_edited.shape:
        # PSNR (Peak Signal-to-Noise Ratio)
        mse = np.mean((arr_original - arr_edited) ** 2)
        if mse == 0:
            psnr = float('inf')
        else:
            max_pixel = 255.0
            psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        
        # Différence absolue moyenne
        mae = np.mean(np.abs(arr_original - arr_edited))
        
        # Afficher métriques
        metrics_text = f"📊 Métriques:\n"
        metrics_text += f"   PSNR: {psnr:.2f} dB\n"
        metrics_text += f"   MAE: {mae:.2f}\n"
        metrics_text += f"   Pixels modifiés: {np.sum(arr_original != arr_edited):,}"
        
        fig.text(0.5, 0.02, metrics_text, ha='center', fontsize=12, 
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Interprétation PSNR
    if show_metrics and arr_original.shape == arr_edited.shape:
        print("\n📈 Interprétation PSNR:")
        if psnr > 40:
            print("   ✅ Excellente qualité (PSNR > 40 dB) - Changements subtils")
        elif psnr > 30:
            print("   ✅ Bonne qualité (PSNR 30-40 dB) - Changements visibles mais contrôlés")
        elif psnr > 20:
            print("   ⚠️  Qualité acceptable (PSNR 20-30 dB) - Changements significatifs")
        else:
            print("   ⚠️  Qualité faible (PSNR < 20 dB) - Changements majeurs")

def create_difference_map(
    original_path: str,
    edited_path: str,
    amplification: float = 5.0
) -> None:
    """Crée une carte visuelle des différences entre 2 images
    
    Args:
        original_path: Chemin image originale
        edited_path: Chemin image éditée
        amplification: Facteur amplification différences pour visibilité
    """
    img_original = np.array(Image.open(original_path))
    img_edited = np.array(Image.open(edited_path))
    
    if img_original.shape != img_edited.shape:
        print("❌ Images de tailles différentes, impossible de comparer")
        return
    
    # Calculer différence absolue
    diff = np.abs(img_original.astype(float) - img_edited.astype(float))
    
    # Amplifier pour visibilité
    diff_amplified = np.clip(diff * amplification, 0, 255).astype(np.uint8)
    
    # Afficher
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    axes[0].imshow(img_original)
    axes[0].set_title("Originale", fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(img_edited)
    axes[1].set_title("Éditée", fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(diff_amplified)
    axes[2].set_title(f"Carte Différences (×{amplification})", fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques différences
    print(f"\n📊 Statistiques différences:")
    print(f"   Moyenne: {np.mean(diff):.2f}")
    print(f"   Max: {np.max(diff):.2f}")
    print(f"   Pixels modifiés (>5): {np.sum(diff > 5):,} ({100*np.sum(diff > 5)/diff.size:.2f}%)")

# ========================================
# EXEMPLE D'UTILISATION
# ========================================

# Cas d'usage: Comparer original vs édition Qwen
# compare_before_after(
#     original_path="cat_original.png",
#     edited_path="cat_with_sunglasses.png",
#     title_original="Chat Original",
#     title_edited="Chat avec Lunettes (Qwen Edit)",
#     show_metrics=True
# )

# Cas d'usage: Carte différences pour analyse détaillée
# create_difference_map(
#     original_path="cat_original.png",
#     edited_path="cat_with_sunglasses.png",
#     amplification=10.0
# )

print("✅ Fonctions comparaison avant/après définies")
print("   - compare_before_after(): Affichage side-by-side + métriques")
print("   - create_difference_map(): Carte visuelle différences")
print("\n💡 Décommentez les exemples ci-dessus pour tester avec vos images")

## ⚙️ Bonnes Pratiques ComfyUI

### 1. Gestion des Erreurs Courantes

#### Timeout (>120s)
**Cause**: GPU surchargé, workflow trop complexe
**Solution**:
```python
client.execute_workflow(workflow, max_wait=300)  # Augmenter timeout
```

#### CUDA Out of Memory
**Cause**: Résolution trop élevée (>768x768)
**Solution**:
- Réduire résolution (512x512 optimal)
- Diminuer `batch_size`
- Simplifier workflow

#### Node Not Found
**Cause**: `class_type` invalide ou custom node manquant
**Solution**: Vérifier documentation custom nodes Qwen

### 2. Optimisation Performance

| Paramètre | Impact Performance | Recommandation |
|-----------|-------------------|----------------|
| **steps** | ↑ steps = ↑ temps | 20-25 optimal |
| **resolution** | ↑ résolution = ↑↑ VRAM | 512x512 par défaut |
| **denoise** | Minimal | N/A |
| **cfg** | Minimal | 7-8 optimal |

### 3. Workflow Reproductible

**Toujours fixer seed pour debugging**:
```python
"seed": 42  # Même résultat à chaque exécution
```

**Seed aléatoire pour variété**:
```python
"seed": int(time.time())  # Différent à chaque fois
```

### 4. Logs et Debugging

**Activer verbose**:
```python
result = client.execute_workflow(workflow, verbose=True)
```

**Inspecter outputs**:
```python
print(json.dumps(result["outputs"], indent=2))
```

In [ ]:
# ========================================
# 🎯 EXERCICE PRATIQUE
# ========================================
# Créez votre propre workflow d'édition d'image

# OBJECTIF:
# Modifier une image en ajoutant un effet spécifique via Qwen VLM

# INSTRUCTIONS:
# 1. Choisissez une image de test (ou utilisez celle générée précédemment)
# 2. Créez un workflow image-to-image avec un prompt créatif
# 3. Testez différentes valeurs de denoise (0.3, 0.5, 0.7)
# 4. Comparez visuellement les résultats

# TODO: Complétez le workflow ci-dessous
workflow_exercice = {
    "5": {
        "class_type": "CheckpointLoaderSimple",
        "inputs": {
            "ckpt_name": "qwen2vl.safetensors"  # Modèle Qwen
        }
    },
    "10": {
        "class_type": "LoadImage",
        "inputs": {
            # TODO: Ajoutez le chemin de votre image source
            "image": "___VOTRE_IMAGE_ICI___"
        }
    },
    "6": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            # TODO: Écrivez un prompt créatif (ex: "convert to watercolor painting")
            "text": "___VOTRE_PROMPT_ICI___",
            "clip": ["5", 1]
        }
    },
    "3": {
        "class_type": "KSampler",
        "inputs": {
            "seed": 42,
            # TODO: Testez différentes valeurs de denoise (0.3 à 0.7)
            "denoise": 0.5,
            "steps": 20,
            "cfg": 7.5,
            "sampler_name": "euler",
            "scheduler": "normal",
            "model": ["5", 0],
            "positive": ["6", 0],
            "negative": ["7", 0],
            "latent_image": ["13", 0]
        }
    },
    "7": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "blurry, low quality, distorted",
            "clip": ["5", 1]
        }
    },
    "13": {
        "class_type": "VAEEncode",
        "inputs": {
            "pixels": ["10", 0],
            "vae": ["5", 2]
        }
    },
    "8": {
        "class_type": "VAEDecode",
        "inputs": {
            "samples": ["3", 0],
            "vae": ["5", 2]
        }
    },
    "9": {
        "class_type": "SaveImage",
        "inputs": {
            "filename_prefix": "exercice_qwen",
            "images": ["8", 0]
        }
    }
}

# AIDE:
# - Pour l'image source: utilisez le nom d'une image uploadée (ex: "cat_512x512.png")
# - Pour le prompt: soyez créatif! (ex: "make it look like a Van Gogh painting")
# - Pour denoise: commencez par 0.5, puis testez 0.3 et 0.7

# BONUS:
# Créez une fonction pour tester plusieurs prompts automatiquement
def tester_prompts_exercice(prompts_list, image_source, denoise=0.5):
    """
    Teste plusieurs prompts sur la même image
    
    Args:
        prompts_list: Liste de prompts à tester
        image_source: Nom de l'image source
        denoise: Valeur denoise (défaut 0.5)
    """
    results = []
    
    for i, prompt in enumerate(prompts_list):
        print(f"\n🎨 Test {i+1}/{len(prompts_list)}: {prompt}")
        
        # TODO: Modifiez le workflow pour chaque prompt
        workflow_test = workflow_exercice.copy()
        # ... (ajoutez votre code ici)
        
        # Exécution workflow
        # result = client.execute_workflow(workflow_test)
        # results.append(result)
    
    return results

# TESTEZ VOTRE CODE:
# prompts_test = [
#     "convert to watercolor painting",
#     "add dramatic sunset lighting",
#     "make it look like a pencil sketch"
# ]
# resultats = tester_prompts_exercice(prompts_test, "cat_512x512.png")

print("\n✅ Exercice prêt! Complétez les TODO et exécutez la cellule.")
print("💡 Conseil: Commencez simple, puis ajoutez de la complexité progressivement.")

## 📚 Ressources Complémentaires

### Documentation Officielle

#### ComfyUI
- **GitHub**: [https://github.com/comfyanonymous/ComfyUI](https://github.com/comfyanonymous/ComfyUI)
- **Documentation API**: [https://github.com/comfyanonymous/ComfyUI/wiki/API](https://github.com/comfyanonymous/ComfyUI/wiki/API)
- **Custom Nodes**: [https://github.com/ltdrdata/ComfyUI-Manager](https://github.com/ltdrdata/ComfyUI-Manager)

#### Qwen Vision-Language Model
- **Paper Officiel**: *Qwen-VL: A Versatile Vision-Language Model*
- **Modèle Hugging Face**: [https://huggingface.co/Qwen/Qwen-VL](https://huggingface.co/Qwen/Qwen-VL)
- **Documentation Technique**: [https://qwenlm.github.io/](https://qwenlm.github.io/)

### Workflows Avancés

#### Workflows ComfyUI Communautaires
- **ComfyUI Workflows Gallery**: [https://comfyworkflows.com/](https://comfyworkflows.com/)
- **CivitAI ComfyUI Section**: [https://civitai.com/tag/comfyui](https://civitai.com/tag/comfyui)

#### Custom Nodes Recommandés
- **ComfyUI-Impact-Pack**: Outils post-processing avancés
- **ComfyUI-AnimateDiff**: Animations et vidéos
- **ComfyUI-ControlNet**: Contrôle spatial précis

### Tutoriels et Guides

#### Débutants
1. **ComfyUI Basics** (YouTube): Introduction complète workflows
2. **Qwen-VL Quick Start**: Guide rapide édition images
3. **JSON Workflows 101**: Comprendre structure workflows

#### Intermédiaires
1. **Advanced Prompting Techniques**: Optimisation prompts Qwen
2. **Workflow Optimization**: Réduire temps génération
3. **Multi-Step Workflows**: Chaînage nodes complexes

#### Avancés
1. **Custom Node Development**: Créer vos propres nodes
2. **API Integration**: Intégrer ComfyUI dans applications
3. **Batch Processing**: Automatisation workflows

### Communauté et Support

- **Discord ComfyUI**: [https://discord.gg/comfyui](https://discord.gg/comfyui)
- **Reddit r/comfyui**: Forum communautaire
- **GitHub Discussions**: Questions techniques

### Ressources MyIA.io

- **Guide APIs Étudiants**: [`GUIDE-APIS-ETUDIANTS.md`](../../../../docs/suivis/genai-image/GUIDE-APIS-ETUDIANTS.md)
- **Workflows Qwen Phase 12C**: [`2025-10-16_12C_architectures-5-workflows-qwen.md`](../../../../docs/genai-suivis/2025-10-16_12C_architectures-5-workflows-qwen.md)
- **Notebook Forge SD-XL**: [`01-4-Forge-SD-XL-Turbo.ipynb`](01-4-Forge-SD-XL-Turbo.ipynb) (API REST similaire)

---

### 🎓 Prochaines Étapes Apprentissage

1. **Maîtriser les bases**: Reproduire tous les exemples de ce notebook
2. **Expérimenter**: Modifier workflows, tester nouveaux prompts
3. **Explorer workflows avancés**: ControlNet, AnimateDiff, Multi-Model
4. **Créer projets personnels**: Application web intégrant API Qwen
5. **Contribuer communauté**: Partager vos workflows innovants

---

**✅ Notebook terminé! Bon apprentissage avec Qwen Image Edit! 🚀**